In [2]:
import geopandas as gpd
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.types import VARCHAR, Float, Integer,TIMESTAMP,JSON,Date
import numpy as np
from shapely import geos  
import fiona  
from fiona.crs import from_epsg,from_string
from shapely.geometry import Polygon, Point
from geopandas import GeoSeries
from pandas import Series,DataFrame
import pyproj
from datetime import datetime,timedelta
from matplotlib import pyplot as plt
from shapely.geometry import Point, LineString, shape,MultiLineString
#import networkx as nx
import osmnx as ox
import time
ox.config(log_console=True, use_cache=True)
import folium
import operator
import datetime
from collections import defaultdict
from functools import reduce
import networkx as nx
from scipy.spatial import ConvexHull, convex_hull_plot_2d
import shapestats
from shapely import geometry
import geoplot as gplt


server_ip='140.137.37.194'
engine = create_engine('postgresql://postgres:Co43270131@%s:5432/hyxen' %(server_ip), use_batch_mode=True, pool_size=10, max_overflow=20)

## 找出住家

In [164]:
sql='''
SELECT * FROM chi.lstm_taipei_allurser

'''
lstm_taipei_allurser=gpd.GeoDataFrame.from_postgis(sql, engine, geom_col='geom' )

In [165]:
lstm_taipei_allurser

,lon,ts,device_id,lat,geom,date_part,to_date,km2,hr
0,121.441237,2020-06-13 15:00:53,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,25.021487,POINT (121.44124 25.02149),6.0,2020-06-13,10.131900,0.876852
1,121.441237,2020-06-13 15:00:52,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,25.021487,POINT (121.44124 25.02149),6.0,2020-06-13,10.131900,0.876852
2,121.505587,2020-06-13 13:25:22,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,25.044991,POINT (121.50559 25.04499),6.0,2020-06-13,10.131900,0.876852
3,121.430797,2020-06-13 12:23:03,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,24.981267,POINT (121.43080 24.98127),6.0,2020-06-13,10.131900,0.876852
4,121.321994,2020-05-30 23:34:12,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,24.976814,POINT (121.32199 24.97681),6.0,2020-05-30,10.131900,0.820992
...,...,...,...,...,...,...,...,...,...
78475,121.546859,2019-11-29 12:17:07,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,25.022088,POINT (121.54686 25.02209),5.0,2019-11-29,9.094253,0.812329
78476,121.546859,2019-11-29 12:17:07,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,25.022088,POINT (121.54686 25.02209),5.0,2019-11-29,9.094253,0.812329
78477,121.546859,2019-11-29 12:17:06,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,25.022088,POINT (121.54686 25.02209),5.0,2019-11-29,9.094253,0.812329
78478,121.546872,2019-11-29 11:10:28,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,25.022223,POINT (121.54687 25.02222),5.0,2019-11-29,9.094253,0.812329


In [40]:
lstm_taipei_allurser['device_id'].value_counts()

S1jG1ed8rMa3QGHOMzVurbD5g0Ij/hbU9x6XjF1vUcrbcsJIwCJCJ3NUqlsVMWvh    734
U7Mg2c7t6X8hZ3+pQRXGxUD36QVPfJePGIRmshsWeUDl2P7AfBILMcSo1PQT7E9o    662
JNg3jEHB/Gzp75a/4CTjwqsvtbeftuaqyB3FE3ptOYgGZlBUCnHGvDYd/xbtQ0c0    649
jFqZt0IIeUsC1M1nbBTHcDHYxBfi3Ji7oQIi8VeJAfC+NlMWFQ2QSarWiO3azYlP    605
fj6yiaFZ1Pp6SwCN3xPqsB8LRQ9EdAwwnd0buEIKykpLQ9XTjpCkbo95MJrz++vX    598
                                                                   ... 
7NCsMSBayUdurTd1il3INyYpHhZRV11IbzippF4/5NZWu7/2NzgzFePsa8MqBvGz      6
6MDfE8r6r3RW36z15x24jwQAD2SuDNzaIPOGpv0mWwys5LSJhuhFwzdYrhyfcwIa      6
FSCTpvwSZ254QXL3lZvt3H8hNgx2gzaE1MVOewB7EUsBXv8JyWb+0jrEZeb2iQRK      4
g/WtRJu5UW6YG9eA58qES1mdd9ZnDW+M20iBVNHOGJE6Vm+ta3rSVeIfnViNUBBY      4
/G6iAiz0ckb7ncISLJIwLVAsB4Azrxm81JnsNXio2iL31qhKnyAhJPfHYYbiaZuM      3
Name: device_id, Length: 292, dtype: int64

In [26]:
lstm_taipei_allurser=lstm_taipei_allurser[lstm_taipei_allurser['date_part']!=6.0]
lstm_taipei_allurser=lstm_taipei_allurser[lstm_taipei_allurser['date_part']!=0.0]
lstm_taipei_allurser

,lon,ts,device_id,lat,geom,date_part,to_date,km2,hr
22,121.328017,2020-03-30 23:33:03,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,24.936203,POINT (121.32802 24.93620),1.0,2020-03-30,10.131900,0.834444
23,121.333494,2020-03-30 18:01:52,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,24.936544,POINT (121.33349 24.93654),1.0,2020-03-30,10.131900,0.834444
24,121.354884,2020-03-30 17:31:54,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,24.955078,POINT (121.35488 24.95508),1.0,2020-03-30,10.131900,0.834444
25,121.322173,2020-03-30 17:01:50,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,24.962465,POINT (121.32217 24.96246),1.0,2020-03-30,10.131900,0.834444
26,121.236902,2020-03-30 16:36:02,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,25.080116,POINT (121.23690 25.08012),1.0,2020-03-30,10.131900,0.834444
...,...,...,...,...,...,...,...,...,...
78475,121.546859,2019-11-29 12:17:07,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,25.022088,POINT (121.54686 25.02209),5.0,2019-11-29,9.094253,0.812329
78476,121.546859,2019-11-29 12:17:07,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,25.022088,POINT (121.54686 25.02209),5.0,2019-11-29,9.094253,0.812329
78477,121.546859,2019-11-29 12:17:06,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,25.022088,POINT (121.54686 25.02209),5.0,2019-11-29,9.094253,0.812329
78478,121.546872,2019-11-29 11:10:28,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,25.022223,POINT (121.54687 25.02222),5.0,2019-11-29,9.094253,0.812329


In [27]:
hm=lstm_taipei_allurser["ts"].dt.strftime('%H:%M:%S')
lstm_taipei_allurser['hm']=hm
lstm_taipei_allurser

,lon,ts,device_id,lat,geom,date_part,to_date,km2,hr,hm
22,121.328017,2020-03-30 23:33:03,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,24.936203,POINT (121.32802 24.93620),1.0,2020-03-30,10.131900,0.834444,23:33:03
23,121.333494,2020-03-30 18:01:52,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,24.936544,POINT (121.33349 24.93654),1.0,2020-03-30,10.131900,0.834444,18:01:52
24,121.354884,2020-03-30 17:31:54,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,24.955078,POINT (121.35488 24.95508),1.0,2020-03-30,10.131900,0.834444,17:31:54
25,121.322173,2020-03-30 17:01:50,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,24.962465,POINT (121.32217 24.96246),1.0,2020-03-30,10.131900,0.834444,17:01:50
26,121.236902,2020-03-30 16:36:02,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,25.080116,POINT (121.23690 25.08012),1.0,2020-03-30,10.131900,0.834444,16:36:02
...,...,...,...,...,...,...,...,...,...,...
78475,121.546859,2019-11-29 12:17:07,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,25.022088,POINT (121.54686 25.02209),5.0,2019-11-29,9.094253,0.812329,12:17:07
78476,121.546859,2019-11-29 12:17:07,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,25.022088,POINT (121.54686 25.02209),5.0,2019-11-29,9.094253,0.812329,12:17:07
78477,121.546859,2019-11-29 12:17:06,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,25.022088,POINT (121.54686 25.02209),5.0,2019-11-29,9.094253,0.812329,12:17:06
78478,121.546872,2019-11-29 11:10:28,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,25.022223,POINT (121.54687 25.02222),5.0,2019-11-29,9.094253,0.812329,11:10:28


In [28]:
workday2200_0500_1=lstm_taipei_allurser[lstm_taipei_allurser['hm'].between("00:00:00","05:00:00")]
workday2300_0500_2=lstm_taipei_allurser[lstm_taipei_allurser['hm'].between("22:00:00","24:00:00")]
workday2300_0500=pd.concat([workday2200_0500_1,workday2300_0500_2])
workday2300_0500=workday2300_0500.reset_index(drop=True)
workday2300_0500

,lon,ts,device_id,lat,geom,date_part,to_date,km2,hr,hm
0,120.539903,2019-12-17 04:34:54,/JewJWCUYnGBX2qnABrAv8ZzU/Wmogl4YuOSZ0iNm2sXy7...,24.065279,POINT (120.53990 24.06528),2.0,2019-12-17,10.358009,0.913186,04:34:54
1,120.539903,2019-12-17 04:04:16,/JewJWCUYnGBX2qnABrAv8ZzU/Wmogl4YuOSZ0iNm2sXy7...,24.065279,POINT (120.53990 24.06528),2.0,2019-12-17,10.358009,0.913186,04:04:16
2,120.539903,2019-12-17 03:32:32,/JewJWCUYnGBX2qnABrAv8ZzU/Wmogl4YuOSZ0iNm2sXy7...,24.065279,POINT (120.53990 24.06528),2.0,2019-12-17,10.358009,0.913186,03:32:32
3,120.539903,2019-12-17 03:02:34,/JewJWCUYnGBX2qnABrAv8ZzU/Wmogl4YuOSZ0iNm2sXy7...,24.065279,POINT (120.53990 24.06528),2.0,2019-12-17,10.358009,0.913186,03:02:34
4,120.539903,2019-12-17 02:32:36,/JewJWCUYnGBX2qnABrAv8ZzU/Wmogl4YuOSZ0iNm2sXy7...,24.065279,POINT (120.53990 24.06528),2.0,2019-12-17,10.358009,0.913186,02:32:36
...,...,...,...,...,...,...,...,...,...,...
5248,121.508443,2020-02-26 22:12:14,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,25.015590,POINT (121.50844 25.01559),3.0,2020-02-26,9.094253,0.784722,22:12:14
5249,121.606694,2020-02-25 23:35:27,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,24.027912,POINT (121.60669 24.02791),2.0,2020-02-25,9.094253,0.909167,23:35:27
5250,121.606466,2020-02-25 22:34:06,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,24.028188,POINT (121.60647 24.02819),2.0,2020-02-25,9.094253,0.909167,22:34:06
5251,121.508478,2019-12-16 23:51:18,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,25.015586,POINT (121.50848 25.01559),1.0,2019-12-16,9.094253,0.696784,23:51:18


In [29]:
workday2300_0500_drop=workday2300_0500.drop_duplicates(inplace=False)#刪重複#原來有2615筆
workday2300_0500_drop

,lon,ts,device_id,lat,geom,date_part,to_date,km2,hr,hm
0,120.539903,2019-12-17 04:34:54,/JewJWCUYnGBX2qnABrAv8ZzU/Wmogl4YuOSZ0iNm2sXy7...,24.065279,POINT (120.53990 24.06528),2.0,2019-12-17,10.358009,0.913186,04:34:54
1,120.539903,2019-12-17 04:04:16,/JewJWCUYnGBX2qnABrAv8ZzU/Wmogl4YuOSZ0iNm2sXy7...,24.065279,POINT (120.53990 24.06528),2.0,2019-12-17,10.358009,0.913186,04:04:16
2,120.539903,2019-12-17 03:32:32,/JewJWCUYnGBX2qnABrAv8ZzU/Wmogl4YuOSZ0iNm2sXy7...,24.065279,POINT (120.53990 24.06528),2.0,2019-12-17,10.358009,0.913186,03:32:32
3,120.539903,2019-12-17 03:02:34,/JewJWCUYnGBX2qnABrAv8ZzU/Wmogl4YuOSZ0iNm2sXy7...,24.065279,POINT (120.53990 24.06528),2.0,2019-12-17,10.358009,0.913186,03:02:34
4,120.539903,2019-12-17 02:32:36,/JewJWCUYnGBX2qnABrAv8ZzU/Wmogl4YuOSZ0iNm2sXy7...,24.065279,POINT (120.53990 24.06528),2.0,2019-12-17,10.358009,0.913186,02:32:36
...,...,...,...,...,...,...,...,...,...,...
5248,121.508443,2020-02-26 22:12:14,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,25.015590,POINT (121.50844 25.01559),3.0,2020-02-26,9.094253,0.784722,22:12:14
5249,121.606694,2020-02-25 23:35:27,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,24.027912,POINT (121.60669 24.02791),2.0,2020-02-25,9.094253,0.909167,23:35:27
5250,121.606466,2020-02-25 22:34:06,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,24.028188,POINT (121.60647 24.02819),2.0,2020-02-25,9.094253,0.909167,22:34:06
5251,121.508478,2019-12-16 23:51:18,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,25.015586,POINT (121.50848 25.01559),1.0,2019-12-16,9.094253,0.696784,23:51:18


In [34]:
def shapestats_area(df):
    if len(df)>2:
        _df=df[['lon','lat']].values.mean(axis=0)
        #print(_df)
        shape = geometry.Point(_df)
        
        #try:
            #area=shapestats.maxbc.maximum_contained_circle(shape).area
        #except:
            #area=0
        return shape
workday2300_0500_drop_group=workday2300_0500_drop.groupby(['device_id','to_date']).apply(shapestats_area)
#taipei_allurser_groupby=taipei_allurser_weekdays_drop.groupby(['device_id','日期'])
workday2300_0500_drop_group

device_id                                                         to_date   
+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNACDG0Ptk9H9hDb0od1Uo  2020-03-04              POINT (121.360965275 25.04008085)
                                                                  2020-03-06           POINT (121.2849870666667 25.0485159)
                                                                  2020-03-09    POINT (121.3308947333333 24.96323383333333)
                                                                  2020-03-10                                           None
                                                                  2020-03-13               POINT (121.2726078 25.025150475)
                                                                                                   ...                     
ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2hqx9gHC+kpB8PvvyWC  2020-03-26          POINT (121.5085609 25.01549913333333)
                                                       

In [35]:
workday2300_0500_drop_group=workday2300_0500_drop_group.add_suffix('').reset_index()
workday2300_0500_drop_group

,device_id,to_date,0
0,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,2020-03-04,POINT (121.360965275 25.04008085)
1,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,2020-03-06,POINT (121.2849870666667 25.0485159)
2,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,2020-03-09,POINT (121.3308947333333 24.96323383333333)
3,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,2020-03-10,None
4,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,2020-03-13,POINT (121.2726078 25.025150475)
...,...,...,...
1653,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,2020-03-26,POINT (121.5085609 25.01549913333333)
1654,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,2020-03-27,POINT (121.5085715333333 25.01549253333333)
1655,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,2020-03-30,None
1656,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,2020-09-10,POINT (121.50853978 25.01551976)


In [36]:
workday2300_0500_drop_group.rename(columns={0:'home'}, inplace = True)

In [37]:
#print(workday2300_0500_drop[workday2300_0500_drop.isnull().T.any()])
workday2300_0500_drop_group.dropna(axis=0, how='any', inplace=True)#刪除空值的欄位

In [38]:
#使用者住家的位置
workday2300_0500_drop_group

,device_id,to_date,home
0,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,2020-03-04,POINT (121.360965275 25.04008085)
1,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,2020-03-06,POINT (121.2849870666667 25.0485159)
2,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,2020-03-09,POINT (121.3308947333333 24.96323383333333)
4,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,2020-03-13,POINT (121.2726078 25.025150475)
5,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,2020-03-18,POINT (121.208332725 24.91374945)
...,...,...,...
1651,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,2020-03-24,POINT (121.508536375 25.015523925)
1652,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,2020-03-25,POINT (121.5084868333333 25.01556536666666)
1653,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,2020-03-26,POINT (121.5085609 25.01549913333333)
1654,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,2020-03-27,POINT (121.5085715333333 25.01549253333333)


In [39]:
workday2300_0500_drop_group['device_id'].value_counts()

4z0UZ2Y0szexv5umcle/DfppEZhkBe5dnFIxOa006I0AM6Sp8xDbwhyKo2y1/3/6    24
O7fpUvu7CGmDtQPpBYYtMK7dW9mF9l94/M+UjhXRtJLBnJYTTpHxZN/+YyxQ2Eaw    22
yqP82rD3/2KVN7af93rq7bt6TpG3HrOdJn03vYIongNP23z7TTj3f1q3oNMFZDzF    19
U7Mg2c7t6X8hZ3+pQRXGxUD36QVPfJePGIRmshsWeUDl2P7AfBILMcSo1PQT7E9o    17
S1jG1ed8rMa3QGHOMzVurbD5g0Ij/hbU9x6XjF1vUcrbcsJIwCJCJ3NUqlsVMWvh    15
                                                                    ..
iP3cf36QckJ1Vmmrz8yF0i4zjuab3BHVjLQebeTTHQtXqkZTrr5tM/O+jSr0xD98     1
/9WkCBjLGEbz+97G+Q1TTn40hzxyEm6DXoSzHcDI8q+0uVnrlYmElQagvA+GoMBw     1
gZOS0CmkmBip+zqFixqDAWwrfv035Uv0W0eskyyKXW6FoxNIACy/A+a16iWtYe8X     1
CZVQzfzVZxMHryZAG0csPH6fKLo6erndG7Dai/3DJd3w9FbMGqKOkDlj0faBJ1z3     1
Nnt04v/D67z/QS5OISm3Nz+zf0x+s5p2fs2jsMiSJtRRivnehPch6XndVZlNxIJ0     1
Name: device_id, Length: 210, dtype: int64

## 迴轉半徑

In [166]:
lstm_taipei_allurser

,lon,ts,device_id,lat,geom,date_part,to_date,km2,hr
0,121.441237,2020-06-13 15:00:53,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,25.021487,POINT (121.44124 25.02149),6.0,2020-06-13,10.131900,0.876852
1,121.441237,2020-06-13 15:00:52,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,25.021487,POINT (121.44124 25.02149),6.0,2020-06-13,10.131900,0.876852
2,121.505587,2020-06-13 13:25:22,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,25.044991,POINT (121.50559 25.04499),6.0,2020-06-13,10.131900,0.876852
3,121.430797,2020-06-13 12:23:03,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,24.981267,POINT (121.43080 24.98127),6.0,2020-06-13,10.131900,0.876852
4,121.321994,2020-05-30 23:34:12,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,24.976814,POINT (121.32199 24.97681),6.0,2020-05-30,10.131900,0.820992
...,...,...,...,...,...,...,...,...,...
78475,121.546859,2019-11-29 12:17:07,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,25.022088,POINT (121.54686 25.02209),5.0,2019-11-29,9.094253,0.812329
78476,121.546859,2019-11-29 12:17:07,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,25.022088,POINT (121.54686 25.02209),5.0,2019-11-29,9.094253,0.812329
78477,121.546859,2019-11-29 12:17:06,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,25.022088,POINT (121.54686 25.02209),5.0,2019-11-29,9.094253,0.812329
78478,121.546872,2019-11-29 11:10:28,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,25.022223,POINT (121.54687 25.02222),5.0,2019-11-29,9.094253,0.812329


In [167]:
lstm_taipei_allurser=lstm_taipei_allurser.to_crs(epsg=3826)
lstm_taipei_allurser["x"] = lstm_taipei_allurser.centroid.x
lstm_taipei_allurser["y"] = lstm_taipei_allurser.centroid.y
#lstm_taipei_allurser["日期"] = lstm_taipei_allurser['ts'].apply(toDate)
lstm_taipei_allurser=lstm_taipei_allurser.reset_index(drop=True)
lstm_taipei_allurser=lstm_taipei_allurser.drop(columns=['lon', 'lat','geom'])
lstm_taipei_allurser=lstm_taipei_allurser.drop_duplicates(inplace=False)#刪重複#原來有11795892筆
lstm_taipei_allurser

,ts,device_id,date_part,to_date,km2,hr,x,y
0,2020-06-13 15:00:53,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,6.0,2020-06-13,10.131900,0.876852,294531.037010,2.768230e+06
1,2020-06-13 15:00:52,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,6.0,2020-06-13,10.131900,0.876852,294531.037010,2.768230e+06
2,2020-06-13 13:25:22,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,6.0,2020-06-13,10.131900,0.876852,301015.777383,2.770856e+06
3,2020-06-13 12:23:03,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,6.0,2020-06-13,10.131900,0.876852,293491.513584,2.763772e+06
4,2020-05-30 23:34:12,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,6.0,2020-05-30,10.131900,0.820992,282508.290690,2.763248e+06
...,...,...,...,...,...,...,...,...
78474,2019-11-29 12:32:04,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,5.0,2019-11-29,9.094253,0.812329,305190.808079,2.768350e+06
78475,2019-11-29 12:17:07,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,5.0,2019-11-29,9.094253,0.812329,305190.664586,2.768336e+06
78477,2019-11-29 12:17:06,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,5.0,2019-11-29,9.094253,0.812329,305190.664586,2.768336e+06
78478,2019-11-29 11:10:28,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,5.0,2019-11-29,9.094253,0.812329,305191.886141,2.768350e+06


In [168]:
def shapestats_area(df):
    if len(df)>2:
        _df=df[['x','y']].values.tolist()
        shape = geometry.Polygon(_df).simplify(0)
        xy = shapestats.compactness._get_pointset(shape)
        try:
            area=shapestats.maxbc.maximum_contained_circle(xy)
            area_=shapestats.maxbc.maximum_contained_circle(shape).area
            radius=area[1]
            center=area[0]
            
        except:
            radius=0
            center=0
            area_=0
        #break
        return [radius,center,area_]


ALL=lstm_taipei_allurser.groupby(['device_id','to_date']).apply(shapestats_area)

In [169]:
#shapestats.maxbc.maximum_contained_circle(xy)
ALL

device_id                                                         to_date   
+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNACDG0Ptk9H9hDb0od1Uo  2020-02-24    [(275615.282273069, 2769250.7677608803), 3383....
                                                                  2020-03-01    [(285061.897418615, 2762656.415086719), 1979.8...
                                                                  2020-03-04    [(281036.83386307984, 2770712.0812995993), 287...
                                                                  2020-03-06    [(279234.1029349184, 2769410.7866373993), 864....
                                                                  2020-03-09    [(292542.15181574493, 2765909.6170811616), 403...
                                                                                                      ...                        
ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2hqx9gHC+kpB8PvvyWC  2020-08-10                              [(nan, nan), -inf, 0.0]
             

In [170]:
ALL=ALL.add_suffix('').reset_index()
ALL

,device_id,to_date,0
0,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,2020-02-24,"[(275615.282273069, 2769250.7677608803), 3383...."
1,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,2020-03-01,"[(285061.897418615, 2762656.415086719), 1979.8..."
2,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,2020-03-04,"[(281036.83386307984, 2770712.0812995993), 287..."
3,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,2020-03-06,"[(279234.1029349184, 2769410.7866373993), 864...."
4,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,2020-03-09,"[(292542.15181574493, 2765909.6170811616), 403..."
...,...,...,...
5632,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,2020-08-10,"[(nan, nan), -inf, 0.0]"
5633,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,2020-08-12,"[(nan, nan), -inf, 0.0]"
5634,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,2020-08-26,"[(nan, nan), -inf, 0.0]"
5635,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,2020-09-10,"[(301329.39272735675, 2767593.8430754533), 1.1..."


In [171]:
ALL

,device_id,to_date,0
0,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,2020-02-24,"[(275615.282273069, 2769250.7677608803), 3383...."
1,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,2020-03-01,"[(285061.897418615, 2762656.415086719), 1979.8..."
2,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,2020-03-04,"[(281036.83386307984, 2770712.0812995993), 287..."
3,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,2020-03-06,"[(279234.1029349184, 2769410.7866373993), 864...."
4,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,2020-03-09,"[(292542.15181574493, 2765909.6170811616), 403..."
...,...,...,...
5632,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,2020-08-10,"[(nan, nan), -inf, 0.0]"
5633,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,2020-08-12,"[(nan, nan), -inf, 0.0]"
5634,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,2020-08-26,"[(nan, nan), -inf, 0.0]"
5635,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,2020-09-10,"[(301329.39272735675, 2767593.8430754533), 1.1..."


In [172]:
ALL=ALL.rename(columns={0:'A'})
ALL

,device_id,to_date,A
0,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,2020-02-24,"[(275615.282273069, 2769250.7677608803), 3383...."
1,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,2020-03-01,"[(285061.897418615, 2762656.415086719), 1979.8..."
2,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,2020-03-04,"[(281036.83386307984, 2770712.0812995993), 287..."
3,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,2020-03-06,"[(279234.1029349184, 2769410.7866373993), 864...."
4,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,2020-03-09,"[(292542.15181574493, 2765909.6170811616), 403..."
...,...,...,...
5632,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,2020-08-10,"[(nan, nan), -inf, 0.0]"
5633,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,2020-08-12,"[(nan, nan), -inf, 0.0]"
5634,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,2020-08-26,"[(nan, nan), -inf, 0.0]"
5635,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,2020-09-10,"[(301329.39272735675, 2767593.8430754533), 1.1..."


In [173]:
ALL['center']=ALL.A.str[0]
ALL['radius']= ALL.A.str[1]
ALL['area']= ALL.A.str[2]
ALL

,device_id,to_date,A,center,radius,area
0,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,2020-02-24,"[(275615.282273069, 2769250.7677608803), 3383....","(275615.282273069, 2769250.7677608803)",3383.320439,3.590362e+07
1,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,2020-03-01,"[(285061.897418615, 2762656.415086719), 1979.8...","(285061.897418615, 2762656.415086719)",1979.875244,1.229498e+07
2,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,2020-03-04,"[(281036.83386307984, 2770712.0812995993), 287...","(281036.83386307984, 2770712.0812995993)",2875.276084,2.593051e+07
3,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,2020-03-06,"[(279234.1029349184, 2769410.7866373993), 864....","(279234.1029349184, 2769410.7866373993)",864.862316,2.346097e+06
4,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,2020-03-09,"[(292542.15181574493, 2765909.6170811616), 403...","(292542.15181574493, 2765909.6170811616)",403.838304,5.115252e+05
...,...,...,...,...,...,...
5632,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,2020-08-10,"[(nan, nan), -inf, 0.0]","(nan, nan)",-inf,0.000000e+00
5633,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,2020-08-12,"[(nan, nan), -inf, 0.0]","(nan, nan)",-inf,0.000000e+00
5634,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,2020-08-26,"[(nan, nan), -inf, 0.0]","(nan, nan)",-inf,0.000000e+00
5635,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,2020-09-10,"[(301329.39272735675, 2767593.8430754533), 1.1...","(301329.39272735675, 2767593.8430754533)",1.168637,4.283627e+00


In [174]:
ALL=ALL.drop(['A'], axis=1) #利用drop方法将含有特定数值的列删除
ALL

,device_id,to_date,center,radius,area
0,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,2020-02-24,"(275615.282273069, 2769250.7677608803)",3383.320439,3.590362e+07
1,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,2020-03-01,"(285061.897418615, 2762656.415086719)",1979.875244,1.229498e+07
2,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,2020-03-04,"(281036.83386307984, 2770712.0812995993)",2875.276084,2.593051e+07
3,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,2020-03-06,"(279234.1029349184, 2769410.7866373993)",864.862316,2.346097e+06
4,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,2020-03-09,"(292542.15181574493, 2765909.6170811616)",403.838304,5.115252e+05
...,...,...,...,...,...
5632,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,2020-08-10,"(nan, nan)",-inf,0.000000e+00
5633,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,2020-08-12,"(nan, nan)",-inf,0.000000e+00
5634,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,2020-08-26,"(nan, nan)",-inf,0.000000e+00
5635,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,2020-09-10,"(301329.39272735675, 2767593.8430754533)",1.168637,4.283627e+00


In [175]:
#ALL['radius'].replace(['-inf'], 0)
ALL=ALL[ALL['radius']>0]
ALL

,device_id,to_date,center,radius,area
0,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,2020-02-24,"(275615.282273069, 2769250.7677608803)",3383.320439,3.590362e+07
1,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,2020-03-01,"(285061.897418615, 2762656.415086719)",1979.875244,1.229498e+07
2,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,2020-03-04,"(281036.83386307984, 2770712.0812995993)",2875.276084,2.593051e+07
3,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,2020-03-06,"(279234.1029349184, 2769410.7866373993)",864.862316,2.346097e+06
4,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,2020-03-09,"(292542.15181574493, 2765909.6170811616)",403.838304,5.115252e+05
...,...,...,...,...,...
5624,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,2020-03-30,"(303698.71834004624, 2767973.169085186)",146.214268,6.705505e+04
5625,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,2020-04-16,"(301325.361888974, 2767592.748476129)",0.146211,6.705227e-02
5631,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,2020-07-29,"(301326.7860072218, 2767591.7257920946)",0.080055,2.010165e-02
5635,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,2020-09-10,"(301329.39272735675, 2767593.8430754533)",1.168637,4.283627e+00


In [176]:
#移動範圍
#radius公尺
ALL["km2"] =(ALL['area']/1000000.0)
ALL

<ipython-input-176-da7765df538d>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ALL["km2"] =(ALL['area']/1000000.0)


,device_id,to_date,center,radius,area,km2
0,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,2020-02-24,"(275615.282273069, 2769250.7677608803)",3383.320439,3.590362e+07,3.590362e+01
1,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,2020-03-01,"(285061.897418615, 2762656.415086719)",1979.875244,1.229498e+07,1.229498e+01
2,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,2020-03-04,"(281036.83386307984, 2770712.0812995993)",2875.276084,2.593051e+07,2.593051e+01
3,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,2020-03-06,"(279234.1029349184, 2769410.7866373993)",864.862316,2.346097e+06,2.346097e+00
4,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,2020-03-09,"(292542.15181574493, 2765909.6170811616)",403.838304,5.115252e+05,5.115252e-01
...,...,...,...,...,...,...
5624,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,2020-03-30,"(303698.71834004624, 2767973.169085186)",146.214268,6.705505e+04,6.705505e-02
5625,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,2020-04-16,"(301325.361888974, 2767592.748476129)",0.146211,6.705227e-02,6.705227e-08
5631,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,2020-07-29,"(301326.7860072218, 2767591.7257920946)",0.080055,2.010165e-02,2.010165e-08
5635,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,2020-09-10,"(301329.39272735675, 2767593.8430754533)",1.168637,4.283627e+00,4.283627e-06


### 移動頻率

In [144]:
sql='''
SELECT * FROM chi.lstm_taipei_allurser

'''
lstm_taipei_allurser=gpd.GeoDataFrame.from_postgis(sql, engine, geom_col='geom' )

In [145]:
#移動頻率
lstm_taipei_allurser

,lon,ts,device_id,lat,geom,date_part,to_date,km2,hr
0,121.441237,2020-06-13 15:00:53,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,25.021487,POINT (121.44124 25.02149),6.0,2020-06-13,10.131900,0.876852
1,121.441237,2020-06-13 15:00:52,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,25.021487,POINT (121.44124 25.02149),6.0,2020-06-13,10.131900,0.876852
2,121.505587,2020-06-13 13:25:22,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,25.044991,POINT (121.50559 25.04499),6.0,2020-06-13,10.131900,0.876852
3,121.430797,2020-06-13 12:23:03,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,24.981267,POINT (121.43080 24.98127),6.0,2020-06-13,10.131900,0.876852
4,121.321994,2020-05-30 23:34:12,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,24.976814,POINT (121.32199 24.97681),6.0,2020-05-30,10.131900,0.820992
...,...,...,...,...,...,...,...,...,...
78475,121.546859,2019-11-29 12:17:07,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,25.022088,POINT (121.54686 25.02209),5.0,2019-11-29,9.094253,0.812329
78476,121.546859,2019-11-29 12:17:07,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,25.022088,POINT (121.54686 25.02209),5.0,2019-11-29,9.094253,0.812329
78477,121.546859,2019-11-29 12:17:06,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,25.022088,POINT (121.54686 25.02209),5.0,2019-11-29,9.094253,0.812329
78478,121.546872,2019-11-29 11:10:28,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,25.022223,POINT (121.54687 25.02222),5.0,2019-11-29,9.094253,0.812329


In [146]:
lstm_taipei_allurser=lstm_taipei_allurser.to_crs(epsg=3826)
lstm_taipei_allurser["x"] = lstm_taipei_allurser.centroid.x
lstm_taipei_allurser["y"] = lstm_taipei_allurser.centroid.y
#lstm_taipei_allurser["日期"] = lstm_taipei_allurser['ts'].apply(toDate)
lstm_taipei_allurser=lstm_taipei_allurser.reset_index(drop=True)
lstm_taipei_allurser=lstm_taipei_allurser.drop(columns=['lon', 'lat','geom'])
lstm_taipei_allurser=lstm_taipei_allurser.drop_duplicates(inplace=False)#刪重複#原來有11795892筆
lstm_taipei_allurser

,ts,device_id,date_part,to_date,km2,hr,x,y
0,2020-06-13 15:00:53,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,6.0,2020-06-13,10.131900,0.876852,294531.037010,2.768230e+06
1,2020-06-13 15:00:52,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,6.0,2020-06-13,10.131900,0.876852,294531.037010,2.768230e+06
2,2020-06-13 13:25:22,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,6.0,2020-06-13,10.131900,0.876852,301015.777383,2.770856e+06
3,2020-06-13 12:23:03,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,6.0,2020-06-13,10.131900,0.876852,293491.513584,2.763772e+06
4,2020-05-30 23:34:12,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,6.0,2020-05-30,10.131900,0.820992,282508.290690,2.763248e+06
...,...,...,...,...,...,...,...,...
78474,2019-11-29 12:32:04,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,5.0,2019-11-29,9.094253,0.812329,305190.808079,2.768350e+06
78475,2019-11-29 12:17:07,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,5.0,2019-11-29,9.094253,0.812329,305190.664586,2.768336e+06
78477,2019-11-29 12:17:06,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,5.0,2019-11-29,9.094253,0.812329,305190.664586,2.768336e+06
78478,2019-11-29 11:10:28,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,5.0,2019-11-29,9.094253,0.812329,305191.886141,2.768350e+06


In [147]:
lstm_taipei_allurser = gpd.GeoDataFrame(
    lstm_taipei_allurser, geometry=gpd.points_from_xy(lstm_taipei_allurser.x, lstm_taipei_allurser.y))
lstm_taipei_allurser

,ts,device_id,date_part,to_date,km2,hr,x,y,geometry
0,2020-06-13 15:00:53,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,6.0,2020-06-13,10.131900,0.876852,294531.037010,2.768230e+06,POINT (294531.037 2768229.986)
1,2020-06-13 15:00:52,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,6.0,2020-06-13,10.131900,0.876852,294531.037010,2.768230e+06,POINT (294531.037 2768229.986)
2,2020-06-13 13:25:22,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,6.0,2020-06-13,10.131900,0.876852,301015.777383,2.770856e+06,POINT (301015.777 2770856.195)
3,2020-06-13 12:23:03,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,6.0,2020-06-13,10.131900,0.876852,293491.513584,2.763772e+06,POINT (293491.514 2763771.717)
4,2020-05-30 23:34:12,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,6.0,2020-05-30,10.131900,0.820992,282508.290690,2.763248e+06,POINT (282508.291 2763247.971)
...,...,...,...,...,...,...,...,...,...
78474,2019-11-29 12:32:04,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,5.0,2019-11-29,9.094253,0.812329,305190.808079,2.768350e+06,POINT (305190.808 2768349.979)
78475,2019-11-29 12:17:07,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,5.0,2019-11-29,9.094253,0.812329,305190.664586,2.768336e+06,POINT (305190.665 2768335.523)
78477,2019-11-29 12:17:06,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,5.0,2019-11-29,9.094253,0.812329,305190.664586,2.768336e+06,POINT (305190.665 2768335.523)
78478,2019-11-29 11:10:28,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,5.0,2019-11-29,9.094253,0.812329,305191.886141,2.768350e+06,POINT (305191.886 2768350.438)


In [149]:
def calculate_distance(group):
    group['distance']=group.distance(group.shift())
    return group
#lstm_taipei_allurser['distance']=
lstm_taipei_allurser_distance = lstm_taipei_allurser.groupby(['device_id','to_date']).apply(calculate_distance)
    
    
#lstm_taipei_allurser
#lstm_taipei_allurser

In [150]:
group.distance(group.shift())

77704            NaN
77705       2.049221
77707       1.261738
77709       2.806348
77711    3950.799100
77712     356.453823
77713     409.627820
dtype: float64

In [194]:
lstm_taipei_allurser_distance

,ts,device_id,date_part,to_date,km2,hr,x,y,geometry,distance
0,2020-06-13 15:00:53,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,6.0,2020-06-13,10.131900,0.876852,294531.037010,2.768230e+06,POINT (294531.037 2768229.986),NaN
1,2020-06-13 15:00:52,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,6.0,2020-06-13,10.131900,0.876852,294531.037010,2.768230e+06,POINT (294531.037 2768229.986),0.000000
2,2020-06-13 13:25:22,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,6.0,2020-06-13,10.131900,0.876852,301015.777383,2.770856e+06,POINT (301015.777 2770856.195),6996.343872
3,2020-06-13 12:23:03,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,6.0,2020-06-13,10.131900,0.876852,293491.513584,2.763772e+06,POINT (293491.514 2763771.717),10334.620030
4,2020-05-30 23:34:12,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,6.0,2020-05-30,10.131900,0.820992,282508.290690,2.763248e+06,POINT (282508.291 2763247.971),NaN
...,...,...,...,...,...,...,...,...,...,...
78474,2019-11-29 12:32:04,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,5.0,2019-11-29,9.094253,0.812329,305190.808079,2.768350e+06,POINT (305190.808 2768349.979),1.171511
78475,2019-11-29 12:17:07,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,5.0,2019-11-29,9.094253,0.812329,305190.664586,2.768336e+06,POINT (305190.665 2768335.523),14.456412
78477,2019-11-29 12:17:06,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,5.0,2019-11-29,9.094253,0.812329,305190.664586,2.768336e+06,POINT (305190.665 2768335.523),0.000000
78478,2019-11-29 11:10:28,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,5.0,2019-11-29,9.094253,0.812329,305191.886141,2.768350e+06,POINT (305191.886 2768350.438),14.964140


In [195]:
lstm_taipei_allurser_distance = lstm_taipei_allurser_distance.fillna(0)
lstm_taipei_allurser_distance

,ts,device_id,date_part,to_date,km2,hr,x,y,geometry,distance
0,2020-06-13 15:00:53,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,6.0,2020-06-13,10.131900,0.876852,294531.037010,2.768230e+06,POINT (294531.037 2768229.986),0.000000
1,2020-06-13 15:00:52,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,6.0,2020-06-13,10.131900,0.876852,294531.037010,2.768230e+06,POINT (294531.037 2768229.986),0.000000
2,2020-06-13 13:25:22,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,6.0,2020-06-13,10.131900,0.876852,301015.777383,2.770856e+06,POINT (301015.777 2770856.195),6996.343872
3,2020-06-13 12:23:03,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,6.0,2020-06-13,10.131900,0.876852,293491.513584,2.763772e+06,POINT (293491.514 2763771.717),10334.620030
4,2020-05-30 23:34:12,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,6.0,2020-05-30,10.131900,0.820992,282508.290690,2.763248e+06,POINT (282508.291 2763247.971),0.000000
...,...,...,...,...,...,...,...,...,...,...
78474,2019-11-29 12:32:04,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,5.0,2019-11-29,9.094253,0.812329,305190.808079,2.768350e+06,POINT (305190.808 2768349.979),1.171511
78475,2019-11-29 12:17:07,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,5.0,2019-11-29,9.094253,0.812329,305190.664586,2.768336e+06,POINT (305190.665 2768335.523),14.456412
78477,2019-11-29 12:17:06,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,5.0,2019-11-29,9.094253,0.812329,305190.664586,2.768336e+06,POINT (305190.665 2768335.523),0.000000
78478,2019-11-29 11:10:28,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,5.0,2019-11-29,9.094253,0.812329,305191.886141,2.768350e+06,POINT (305191.886 2768350.438),14.964140


### 方位

In [48]:
from numpy import arctan2,random,sin,cos,degrees

In [190]:
sql='''
SELECT * FROM chi.lstm_taipei_allurser

'''
lstm_taipei_allurser=gpd.GeoDataFrame.from_postgis(sql, engine, geom_col='geom' )

In [191]:
lstm_taipei_allurser=lstm_taipei_allurser.to_crs(epsg=3826)
lstm_taipei_allurser["x"] = lstm_taipei_allurser.centroid.x
lstm_taipei_allurser["y"] = lstm_taipei_allurser.centroid.y
#lstm_taipei_allurser["日期"] = lstm_taipei_allurser['ts'].apply(toDate)
lstm_taipei_allurser=lstm_taipei_allurser.reset_index(drop=True)
lstm_taipei_allurser=lstm_taipei_allurser.drop_duplicates(inplace=False)#刪重複#原來有11795892筆

lstm_taipei_allurser

,lon,ts,device_id,lat,geom,date_part,to_date,km2,hr,x,y
0,121.441237,2020-06-13 15:00:53,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,25.021487,POINT (294531.037 2768229.986),6.0,2020-06-13,10.131900,0.876852,294531.037010,2.768230e+06
1,121.441237,2020-06-13 15:00:52,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,25.021487,POINT (294531.037 2768229.986),6.0,2020-06-13,10.131900,0.876852,294531.037010,2.768230e+06
2,121.505587,2020-06-13 13:25:22,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,25.044991,POINT (301015.777 2770856.195),6.0,2020-06-13,10.131900,0.876852,301015.777383,2.770856e+06
3,121.430797,2020-06-13 12:23:03,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,24.981267,POINT (293491.514 2763771.717),6.0,2020-06-13,10.131900,0.876852,293491.513584,2.763772e+06
4,121.321994,2020-05-30 23:34:12,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,24.976814,POINT (282508.291 2763247.971),6.0,2020-05-30,10.131900,0.820992,282508.290690,2.763248e+06
...,...,...,...,...,...,...,...,...,...,...,...
78474,121.546862,2019-11-29 12:32:04,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,25.022219,POINT (305190.808 2768349.979),5.0,2019-11-29,9.094253,0.812329,305190.808079,2.768350e+06
78475,121.546859,2019-11-29 12:17:07,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,25.022088,POINT (305190.665 2768335.523),5.0,2019-11-29,9.094253,0.812329,305190.664586,2.768336e+06
78477,121.546859,2019-11-29 12:17:06,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,25.022088,POINT (305190.665 2768335.523),5.0,2019-11-29,9.094253,0.812329,305190.664586,2.768336e+06
78478,121.546872,2019-11-29 11:10:28,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,25.022223,POINT (305191.886 2768350.438),5.0,2019-11-29,9.094253,0.812329,305191.886141,2.768350e+06


In [192]:
def position(group):
    group.index = np.arange(1, len(group) + 1)
    #group.reset_index(drop=True, inplace=True)
    #group['position']=group.distance(group.shift())
    #print(group)
    for i in range(1,len(group)):
        try:
            dL=group.lon[i+1]-group.lon[i]
            X = cos(group.lat[i+1])* sin(dL)
            Y = cos(group.lat[i])* sin(group.lat[i+1]) - sin(group.lat[i])*cos(group.lat[i+1])* cos(dL)
            bearing=arctan2(X,Y)
            group['angle'].iloc[i]=degrees(bearing)
            #print(bearing)
        except:
            group['angle'].iloc[i]=np.NaN
    return group

lstm_taipei_allurser['angle'] = np.NaN
lstm_taipei_allurser_position=lstm_taipei_allurser.groupby(['device_id','to_date']).apply(position)


In [193]:
lstm_taipei_allurser_position=lstm_taipei_allurser_position.add_suffix('').reset_index(drop=True)
lstm_taipei_allurser_position

,lon,ts,device_id,lat,geom,date_part,to_date,km2,hr,x,y,angle
0,120.967764,2020-02-24 14:01:33,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,24.820307,POINT (246741.331 2745875.100),1.0,2020-02-24,10.131900,0.875486,246741.331173,2.745875e+06,NaN
1,121.127899,2020-02-24 13:31:31,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,24.894802,POINT (262921.102 2754131.807),1.0,2020-02-24,10.131900,0.875486,262921.102041,2.754132e+06,65.504615
2,121.231670,2020-02-24 13:01:30,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,25.076818,POINT (273370.209 2774306.089),1.0,2020-02-24,10.131900,0.875486,273370.209082,2.774306e+06,29.915297
3,121.256735,2020-02-24 11:01:41,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,25.081010,POINT (275897.858 2774774.967),1.0,2020-02-24,10.131900,0.875486,275897.857969,2.774775e+06,80.531376
4,121.306066,2020-02-24 10:31:26,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,25.016017,POINT (280890.410 2767586.538),1.0,2020-02-24,10.131900,0.875486,280890.409501,2.767587e+06,143.006826
...,...,...,...,...,...,...,...,...,...,...,...,...
73567,121.546812,2020-09-11 15:52:05,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,25.023038,POINT (305185.487 2768440.665),5.0,2020-09-11,9.094253,0.766465,305185.486504,2.768441e+06,78.903590
73568,121.547526,2020-09-11 15:18:53,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,25.019886,POINT (305258.917 2768091.857),5.0,2020-09-11,9.094253,0.766465,305258.917018,2.768092e+06,167.320951
73569,121.546456,2020-09-11 14:46:11,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,25.023453,POINT (305149.291 2768486.543),5.0,2020-09-11,9.094253,0.766465,305149.291100,2.768487e+06,-16.608949
73570,121.546456,2020-09-11 14:32:38,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,25.023453,POINT (305149.291 2768486.543),5.0,2020-09-11,9.094253,0.766465,305149.291100,2.768487e+06,0.000000


### 目前資料
**lstm_taipei_allurser_position : 方位角(73572)**  
**lstm_taipei_allurser_distance : 點與點距離(73572)**  
**workday2300_0500_drop_group : 住家**  
**ALL : 迴轉半徑、移動範圍、中心點**  

In [183]:
from pandas_datareader import data
import matplotlib.pyplot as plt
import pandas as pd
import datetime as dt
import urllib.request, json
import os
import numpy as np
import tensorflow as tf # This code has been tested with TensorFlow 1.6
from sklearn.preprocessing import MinMaxScaler
import numpy
import matplotlib.pyplot as plt
import pandas
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error 

### 有家的使用者

In [312]:
lstm_taipei_allurser_home_center=pd.merge(workday2300_0500_drop_group, ALL, how="inner")
lstm_taipei_allurser_home_center

,device_id,to_date,home,center,radius,area,km2
0,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,2020-03-04,POINT (121.360965275 25.04008085),"(281036.83386307984, 2770712.0812995993)",2875.276084,2.593051e+07,2.593051e+01
1,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,2020-03-06,POINT (121.2849870666667 25.0485159),"(279234.1029349184, 2769410.7866373993)",864.862316,2.346097e+06,2.346097e+00
2,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,2020-03-09,POINT (121.3308947333333 24.96323383333333),"(292542.15181574493, 2765909.6170811616)",403.838304,5.115252e+05,5.115252e-01
3,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,2020-03-13,POINT (121.2726078 25.025150475),"(274665.8445534727, 2773224.6119559235)",92.329244,2.673810e+04,2.673810e-02
4,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,2020-03-18,POINT (121.208332725 24.91374945),"(285038.74204480747, 2767016.4137141635)",3891.603819,4.750171e+07,4.750171e+01
...,...,...,...,...,...,...,...
855,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,2020-03-24,POINT (121.508536375 25.015523925),"(301328.56096520077, 2767592.232771158)",0.780030,1.908421e+00,1.908421e-06
856,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,2020-03-25,POINT (121.5084868333333 25.01556536666666),"(301895.87538888794, 2768133.482767734)",184.670163,1.069659e+05,1.069659e-01
857,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,2020-03-26,POINT (121.5085609 25.01549913333333),"(301329.3697520958, 2767589.436016141)",0.074669,1.748766e-02,1.748766e-08
858,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,2020-03-27,POINT (121.5085715333333 25.01549253333333),"(301332.8321852122, 2767591.4647959014)",0.959602,2.888246e+00,2.888246e-06


In [362]:
lstm_taipei_allurser_home_center["PK"] = lstm_taipei_allurser_home_center["device_id"] + "" + lstm_taipei_allurser_home_center["to_date"]
lstm_taipei_allurser_home_center

,device_id,to_date,home,center,radius,area,km2,PK
0,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,2020-03-04,POINT (121.360965275 25.04008085),"(281036.83386307984, 2770712.0812995993)",2875.276084,2.593051e+07,2.593051e+01,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...
1,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,2020-03-06,POINT (121.2849870666667 25.0485159),"(279234.1029349184, 2769410.7866373993)",864.862316,2.346097e+06,2.346097e+00,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...
2,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,2020-03-09,POINT (121.3308947333333 24.96323383333333),"(292542.15181574493, 2765909.6170811616)",403.838304,5.115252e+05,5.115252e-01,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...
3,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,2020-03-13,POINT (121.2726078 25.025150475),"(274665.8445534727, 2773224.6119559235)",92.329244,2.673810e+04,2.673810e-02,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...
4,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,2020-03-18,POINT (121.208332725 24.91374945),"(285038.74204480747, 2767016.4137141635)",3891.603819,4.750171e+07,4.750171e+01,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...
...,...,...,...,...,...,...,...,...
855,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,2020-03-24,POINT (121.508536375 25.015523925),"(301328.56096520077, 2767592.232771158)",0.780030,1.908421e+00,1.908421e-06,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...
856,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,2020-03-25,POINT (121.5084868333333 25.01556536666666),"(301895.87538888794, 2768133.482767734)",184.670163,1.069659e+05,1.069659e-01,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...
857,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,2020-03-26,POINT (121.5085609 25.01549913333333),"(301329.3697520958, 2767589.436016141)",0.074669,1.748766e-02,1.748766e-08,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...
858,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,2020-03-27,POINT (121.5085715333333 25.01549253333333),"(301332.8321852122, 2767591.4647959014)",0.959602,2.888246e+00,2.888246e-06,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...


In [313]:
lstm_taipei_allurser_position_distance=pd.merge(lstm_taipei_allurser_position, lstm_taipei_allurser_distance, how="inner")
lstm_taipei_allurser_position_distance

,lon,ts,device_id,lat,geom,date_part,to_date,km2,hr,x,y,angle,geometry,distance
0,120.967764,2020-02-24 14:01:33,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,24.820307,POINT (246741.331 2745875.100),1.0,2020-02-24,10.131900,0.875486,246741.331173,2.745875e+06,NaN,POINT (246741.331 2745875.100),0.000000
1,121.127899,2020-02-24 13:31:31,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,24.894802,POINT (262921.102 2754131.807),1.0,2020-02-24,10.131900,0.875486,262921.102041,2.754132e+06,65.504615,POINT (262921.102 2754131.807),18164.751546
2,121.231670,2020-02-24 13:01:30,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,25.076818,POINT (273370.209 2774306.089),1.0,2020-02-24,10.131900,0.875486,273370.209082,2.774306e+06,29.915297,POINT (273370.209 2774306.089),22719.715908
3,121.256735,2020-02-24 11:01:41,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,25.081010,POINT (275897.858 2774774.967),1.0,2020-02-24,10.131900,0.875486,275897.857969,2.774775e+06,80.531376,POINT (275897.858 2774774.967),2570.769435
4,121.306066,2020-02-24 10:31:26,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,25.016017,POINT (280890.410 2767586.538),1.0,2020-02-24,10.131900,0.875486,280890.409501,2.767587e+06,143.006826,POINT (280890.410 2767586.538),8752.090014
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73567,121.546812,2020-09-11 15:52:05,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,25.023038,POINT (305185.487 2768440.665),5.0,2020-09-11,9.094253,0.766465,305185.486504,2.768441e+06,78.903590,POINT (305185.487 2768440.665),3950.799100
73568,121.547526,2020-09-11 15:18:53,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,25.019886,POINT (305258.917 2768091.857),5.0,2020-09-11,9.094253,0.766465,305258.917018,2.768092e+06,167.320951,POINT (305258.917 2768091.857),356.453823
73569,121.546456,2020-09-11 14:46:11,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,25.023453,POINT (305149.291 2768486.543),5.0,2020-09-11,9.094253,0.766465,305149.291100,2.768487e+06,-16.608949,POINT (305149.291 2768486.543),409.627820
73570,121.546456,2020-09-11 14:32:38,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,25.023453,POINT (305149.291 2768486.543),5.0,2020-09-11,9.094253,0.766465,305149.291100,2.768487e+06,0.000000,POINT (305149.291 2768486.543),0.000000


In [357]:
#lstm_taipei_allurser_position_distance=lstm_taipei_allurser_position_distance.drop(['PK'], axis=1)
lstm_taipei_allurser_position_distance['DateStr'] = lstm_taipei_allurser_position_distance['to_date'].apply(lambda x: x.strftime('%Y-%m-%d'))

,lon,ts,device_id,lat,geom,date_part,to_date,km2,hr,x,y,angle,geometry,distance,DateStr
0,120.967764,2020-02-24 14:01:33,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,24.820307,POINT (246741.331 2745875.100),1.0,2020-02-24,10.131900,0.875486,246741.331173,2.745875e+06,NaN,POINT (246741.331 2745875.100),0.000000,2020-02-24
1,121.127899,2020-02-24 13:31:31,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,24.894802,POINT (262921.102 2754131.807),1.0,2020-02-24,10.131900,0.875486,262921.102041,2.754132e+06,65.504615,POINT (262921.102 2754131.807),18164.751546,2020-02-24
2,121.231670,2020-02-24 13:01:30,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,25.076818,POINT (273370.209 2774306.089),1.0,2020-02-24,10.131900,0.875486,273370.209082,2.774306e+06,29.915297,POINT (273370.209 2774306.089),22719.715908,2020-02-24
3,121.256735,2020-02-24 11:01:41,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,25.081010,POINT (275897.858 2774774.967),1.0,2020-02-24,10.131900,0.875486,275897.857969,2.774775e+06,80.531376,POINT (275897.858 2774774.967),2570.769435,2020-02-24
4,121.306066,2020-02-24 10:31:26,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,25.016017,POINT (280890.410 2767586.538),1.0,2020-02-24,10.131900,0.875486,280890.409501,2.767587e+06,143.006826,POINT (280890.410 2767586.538),8752.090014,2020-02-24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73567,121.546812,2020-09-11 15:52:05,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,25.023038,POINT (305185.487 2768440.665),5.0,2020-09-11,9.094253,0.766465,305185.486504,2.768441e+06,78.903590,POINT (305185.487 2768440.665),3950.799100,2020-09-11
73568,121.547526,2020-09-11 15:18:53,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,25.019886,POINT (305258.917 2768091.857),5.0,2020-09-11,9.094253,0.766465,305258.917018,2.768092e+06,167.320951,POINT (305258.917 2768091.857),356.453823,2020-09-11
73569,121.546456,2020-09-11 14:46:11,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,25.023453,POINT (305149.291 2768486.543),5.0,2020-09-11,9.094253,0.766465,305149.291100,2.768487e+06,-16.608949,POINT (305149.291 2768486.543),409.627820,2020-09-11
73570,121.546456,2020-09-11 14:32:38,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,25.023453,POINT (305149.291 2768486.543),5.0,2020-09-11,9.094253,0.766465,305149.291100,2.768487e+06,0.000000,POINT (305149.291 2768486.543),0.000000,2020-09-11


In [387]:
lstm_taipei_allurser_position_distance['PK']=lstm_taipei_allurser_position_distance["device_id"] + "" + lstm_taipei_allurser_position_distance["DateStr"]
lstm_taipei_allurser_position_distance["hr"]=lstm_taipei_allurser_position_distance["ts"].dt.strftime('%H')
lstm_taipei_allurser_position_distance

,lon,ts,device_id,lat,geom,date_part,to_date,km2,hr,x,y,angle,geometry,distance,DateStr,PK
0,120.967764,2020-02-24 14:01:33,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,24.820307,POINT (246741.331 2745875.100),1.0,2020-02-24,10.131900,14,246741.331173,2.745875e+06,NaN,POINT (246741.331 2745875.100),0.000000,2020-02-24,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...
1,121.127899,2020-02-24 13:31:31,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,24.894802,POINT (262921.102 2754131.807),1.0,2020-02-24,10.131900,13,262921.102041,2.754132e+06,65.504615,POINT (262921.102 2754131.807),18164.751546,2020-02-24,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...
2,121.231670,2020-02-24 13:01:30,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,25.076818,POINT (273370.209 2774306.089),1.0,2020-02-24,10.131900,13,273370.209082,2.774306e+06,29.915297,POINT (273370.209 2774306.089),22719.715908,2020-02-24,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...
3,121.256735,2020-02-24 11:01:41,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,25.081010,POINT (275897.858 2774774.967),1.0,2020-02-24,10.131900,11,275897.857969,2.774775e+06,80.531376,POINT (275897.858 2774774.967),2570.769435,2020-02-24,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...
4,121.306066,2020-02-24 10:31:26,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,25.016017,POINT (280890.410 2767586.538),1.0,2020-02-24,10.131900,10,280890.409501,2.767587e+06,143.006826,POINT (280890.410 2767586.538),8752.090014,2020-02-24,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73567,121.546812,2020-09-11 15:52:05,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,25.023038,POINT (305185.487 2768440.665),5.0,2020-09-11,9.094253,15,305185.486504,2.768441e+06,78.903590,POINT (305185.487 2768440.665),3950.799100,2020-09-11,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...
73568,121.547526,2020-09-11 15:18:53,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,25.019886,POINT (305258.917 2768091.857),5.0,2020-09-11,9.094253,15,305258.917018,2.768092e+06,167.320951,POINT (305258.917 2768091.857),356.453823,2020-09-11,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...
73569,121.546456,2020-09-11 14:46:11,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,25.023453,POINT (305149.291 2768486.543),5.0,2020-09-11,9.094253,14,305149.291100,2.768487e+06,-16.608949,POINT (305149.291 2768486.543),409.627820,2020-09-11,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...
73570,121.546456,2020-09-11 14:32:38,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,25.023453,POINT (305149.291 2768486.543),5.0,2020-09-11,9.094253,14,305149.291100,2.768487e+06,0.000000,POINT (305149.291 2768486.543),0.000000,2020-09-11,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...


In [390]:
lstm_home=pd.merge(lstm_taipei_allurser_home_center,
                 lstm_taipei_allurser_position_distance[['device_id','geom','to_date','ts','angle','distance','date_part','PK','hr']],
                 on=['PK'])
lstm_home=lstm_home[['distance','angle','device_id_x','PK','geom','to_date_x','hr','date_part','home','center','radius','km2']]
#lstm_home['device_id_x'].value_counts()
lstm_home


,distance,angle,device_id_x,PK,geom,to_date_x,hr,date_part,home,center,radius,km2
0,0.000000,NaN,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,POINT (285333.598 2767770.953),2020-03-04,23,3.0,POINT (121.360965275 25.04008085),"(281036.83386307984, 2770712.0812995993)",2875.276084,25.930513
1,2176.072253,53.842609,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,POINT (287030.214 2769133.592),2020-03-04,23,3.0,POINT (121.360965275 25.04008085),"(281036.83386307984, 2770712.0812995993)",2875.276084,25.930513
2,2905.662334,-10.450005,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,POINT (286539.710 2771997.554),2020-03-04,22,3.0,POINT (121.360965275 25.04008085),"(281036.83386307984, 2770712.0812995993)",2875.276084,25.930513
3,300.946788,59.739847,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,POINT (286792.895 2772160.239),2020-03-04,22,3.0,POINT (121.360965275 25.04008085),"(281036.83386307984, 2770712.0812995993)",2875.276084,25.930513
4,460.535020,-3.912062,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,POINT (286762.953 2772619.799),2020-03-04,21,3.0,POINT (121.360965275 25.04008085),"(281036.83386307984, 2770712.0812995993)",2875.276084,25.930513
...,...,...,...,...,...,...,...,...,...,...,...,...
18162,0.846538,-97.601023,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,POINT (301328.622 2767590.969),2020-09-10,13,4.0,POINT (121.50853978 25.01551976),"(301329.39272735675, 2767593.8430754533)",1.168637,0.000004
18163,0.245558,-7.712507,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,POINT (301328.591 2767591.213),2020-09-10,12,4.0,POINT (121.50853978 25.01551976),"(301329.39272735675, 2767593.8430754533)",1.168637,0.000004
18164,1.246918,-136.577459,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,POINT (301327.777 2767590.268),2020-09-10,12,4.0,POINT (121.50853978 25.01551976),"(301329.39272735675, 2767593.8430754533)",1.168637,0.000004
18165,0.000000,0.000000,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,POINT (301327.777 2767590.268),2020-09-10,12,4.0,POINT (121.50853978 25.01551976),"(301329.39272735675, 2767593.8430754533)",1.168637,0.000004


### 沒家的使用者

In [391]:
#ALL['PK']=ALL["device_id"] + "" + ALL["to_date"]
lstm_NOT_home=pd.merge(ALL,
                 lstm_taipei_allurser_position_distance[['device_id','geom','to_date','ts','angle','distance','date_part','PK','hr']],
                 on=['PK'])
lstm_NOT_home=lstm_NOT_home[['distance','angle','device_id_x','PK','geom','to_date_x','hr','date_part','center','radius','km2']]
lstm_NOT_home['device_id_x'].value_counts()

S1jG1ed8rMa3QGHOMzVurbD5g0Ij/hbU9x6XjF1vUcrbcsJIwCJCJ3NUqlsVMWvh    1042
IHc6QbRRqVO9E6mfr2XvOysNKgsiz+zp1o5AIdaXZtShbqzryJwWZafHayKeVI0K     861
oQ819+I7XndCq0lQXwfOo8Wd90XSTNzVzSjarzbnFABim8ufEwOq+fVZMMZZINHh     851
fj6yiaFZ1Pp6SwCN3xPqsB8LRQ9EdAwwnd0buEIKykpLQ9XTjpCkbo95MJrz++vX     805
O7fpUvu7CGmDtQPpBYYtMK7dW9mF9l94/M+UjhXRtJLBnJYTTpHxZN/+YyxQ2Eaw     791
                                                                    ... 
mpb3JRYa0MUZl7LFtHmcElt92jMfMBk5FVWGJAqSIMSAM0NN2Behj01bOdHKmsF/       8
ELunFF5x3OSCafc2IfVV5wUjtIMkF7SIpsVVJZcexRqjr4j4DdKmQRLc5AbWClq8       7
7NCsMSBayUdurTd1il3INyYpHhZRV11IbzippF4/5NZWu7/2NzgzFePsa8MqBvGz       6
pC7WW4OguL7y/la32QTui55x7iumEFzMQDPo3d+xqv5T8CuRQFJlD5A7lkEVznli       5
DuD6vmRAQpEFWLUnEI3zYr+abE34XrBEAZJM1v4s55H5xlZvMMrG+zYJM0rVH/Sm       4
Name: device_id_x, Length: 284, dtype: int64

--------------------------------------------------------

## 測試資料
**lstm_home:有家的人**  
**lstm_NOT_home:沒家的人**

In [392]:
lstm_NOT_home

,distance,angle,device_id_x,PK,geom,to_date_x,hr,date_part,center,radius,km2
0,0.000000,NaN,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,POINT (246741.331 2745875.100),2020-02-24,14,1.0,"(275615.282273069, 2769250.7677608803)",3383.320439,35.903623
1,18164.751546,65.504615,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,POINT (262921.102 2754131.807),2020-02-24,13,1.0,"(275615.282273069, 2769250.7677608803)",3383.320439,35.903623
2,22719.715908,29.915297,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,POINT (273370.209 2774306.089),2020-02-24,13,1.0,"(275615.282273069, 2769250.7677608803)",3383.320439,35.903623
3,2570.769435,80.531376,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,POINT (275897.858 2774774.967),2020-02-24,11,1.0,"(275615.282273069, 2769250.7677608803)",3383.320439,35.903623
4,8752.090014,143.006826,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,POINT (280890.410 2767586.538),2020-02-24,10,1.0,"(275615.282273069, 2769250.7677608803)",3383.320439,35.903623
...,...,...,...,...,...,...,...,...,...,...,...
59472,3950.799100,78.903590,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,POINT (305185.487 2768440.665),2020-09-11,15,5.0,"(304806.2732734045, 2768178.70019949)",143.329214,0.064435
59473,356.453823,167.320951,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,POINT (305258.917 2768091.857),2020-09-11,15,5.0,"(304806.2732734045, 2768178.70019949)",143.329214,0.064435
59474,409.627820,-16.608949,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,POINT (305149.291 2768486.543),2020-09-11,14,5.0,"(304806.2732734045, 2768178.70019949)",143.329214,0.064435
59475,0.000000,0.000000,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,POINT (305149.291 2768486.543),2020-09-11,14,5.0,"(304806.2732734045, 2768178.70019949)",143.329214,0.064435


In [428]:
#X_train_cyclic.info()
lstm_NOT_home['hr'] = lstm_NOT_home['hr'].astype(np.float64)
lstm_NOT_home

,distance,angle,device_id_x,PK,geom,to_date_x,hr,date_part,center,radius,km2
0,0.000000,NaN,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,POINT (246741.331 2745875.100),2020-02-24,14.0,1.0,"(275615.282273069, 2769250.7677608803)",3383.320439,35.903623
1,18164.751546,65.504615,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,POINT (262921.102 2754131.807),2020-02-24,13.0,1.0,"(275615.282273069, 2769250.7677608803)",3383.320439,35.903623
2,22719.715908,29.915297,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,POINT (273370.209 2774306.089),2020-02-24,13.0,1.0,"(275615.282273069, 2769250.7677608803)",3383.320439,35.903623
3,2570.769435,80.531376,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,POINT (275897.858 2774774.967),2020-02-24,11.0,1.0,"(275615.282273069, 2769250.7677608803)",3383.320439,35.903623
4,8752.090014,143.006826,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,POINT (280890.410 2767586.538),2020-02-24,10.0,1.0,"(275615.282273069, 2769250.7677608803)",3383.320439,35.903623
...,...,...,...,...,...,...,...,...,...,...,...
59472,3950.799100,78.903590,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,POINT (305185.487 2768440.665),2020-09-11,15.0,5.0,"(304806.2732734045, 2768178.70019949)",143.329214,0.064435
59473,356.453823,167.320951,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,POINT (305258.917 2768091.857),2020-09-11,15.0,5.0,"(304806.2732734045, 2768178.70019949)",143.329214,0.064435
59474,409.627820,-16.608949,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,POINT (305149.291 2768486.543),2020-09-11,14.0,5.0,"(304806.2732734045, 2768178.70019949)",143.329214,0.064435
59475,0.000000,0.000000,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,POINT (305149.291 2768486.543),2020-09-11,14.0,5.0,"(304806.2732734045, 2768178.70019949)",143.329214,0.064435


## 循環特徵編碼

### 住家的循環編碼

In [629]:
lstm_home_cyclic = lstm_home.copy()
columns = ['angle'] # 要操作的两列循环特征
for col in columns:
# 对每个循环特征派生两个特征列，如对于4月，是2pi*4/12，取sin和cos即可
    lstm_home_cyclic[col+'_sin'] = np.sin((2*np.pi*lstm_home_cyclic[col])/180)
    lstm_home_cyclic[col+'_cos'] = np.cos((2*np.pi*lstm_home_cyclic[col])/180)
lstm_home_cyclic=lstm_home_cyclic.drop(columns,axis=1) # 删除原来的循环特征列
lstm_home_cyclic

,distance,device_id_x,PK,geom,to_date_x,hr,date_part,home,center,radius,km2,angle_sin,angle_cos
0,0.000000,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,POINT (285333.598 2767770.953),2020-03-04,23.0,3.0,POINT (121.360965275 25.04008085),"(281036.83386307984, 2770712.0812995993)",2875.276084,25.930513,NaN,NaN
1,2176.072253,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,POINT (287030.214 2769133.592),2020-03-04,23.0,3.0,POINT (121.360965275 25.04008085),"(281036.83386307984, 2770712.0812995993)",2875.276084,25.930513,0.952740,-0.303787
2,2905.662334,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,POINT (286539.710 2771997.554),2020-03-04,22.0,3.0,POINT (121.360965275 25.04008085),"(281036.83386307984, 2770712.0812995993)",2875.276084,25.930513,-0.356738,0.934204
3,300.946788,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,POINT (286792.895 2772160.239),2020-03-04,22.0,3.0,POINT (121.360965275 25.04008085),"(281036.83386307984, 2770712.0812995993)",2875.276084,25.930513,0.870530,-0.492115
4,460.535020,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,POINT (286762.953 2772619.799),2020-03-04,21.0,3.0,POINT (121.360965275 25.04008085),"(281036.83386307984, 2770712.0812995993)",2875.276084,25.930513,-0.136133,0.990691
...,...,...,...,...,...,...,...,...,...,...,...,...,...
18162,0.846538,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,POINT (301328.622 2767590.969),2020-09-10,13.0,4.0,POINT (121.50853978 25.01551976),"(301329.39272735675, 2767593.8430754533)",1.168637,0.000004,0.262224,-0.965007
18163,0.245558,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,POINT (301328.591 2767591.213),2020-09-10,12.0,4.0,POINT (121.50853978 25.01551976),"(301329.39272735675, 2767593.8430754533)",1.168637,0.000004,-0.265977,0.963979
18164,1.246918,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,POINT (301327.777 2767590.268),2020-09-10,12.0,4.0,POINT (121.50853978 25.01551976),"(301329.39272735675, 2767593.8430754533)",1.168637,0.000004,0.998484,0.055036
18165,0.000000,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,POINT (301327.777 2767590.268),2020-09-10,12.0,4.0,POINT (121.50853978 25.01551976),"(301329.39272735675, 2767593.8430754533)",1.168637,0.000004,0.000000,1.000000


In [630]:
columns = ['hr'] # 要操作的两列循环特征
for col in columns:
# 对每个循环特征派生两个特征列，如对于4月，是2pi*4/12，取sin和cos即可
    lstm_home_cyclic[col+'_sin'] = np.sin((2*np.pi*lstm_home_cyclic[col])/23.0)
    lstm_home_cyclic[col+'_cos'] = np.cos((2*np.pi*lstm_home_cyclic[col])/23.0)
lstm_home_cyclic=lstm_home_cyclic.drop(columns,axis=1) # 删除原来的循环特征列
lstm_home_cyclic

,distance,device_id_x,PK,geom,to_date_x,date_part,home,center,radius,km2,angle_sin,angle_cos,hr_sin,hr_cos
0,0.000000,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,POINT (285333.598 2767770.953),2020-03-04,3.0,POINT (121.360965275 25.04008085),"(281036.83386307984, 2770712.0812995993)",2875.276084,25.930513,NaN,NaN,-2.449294e-16,1.000000
1,2176.072253,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,POINT (287030.214 2769133.592),2020-03-04,3.0,POINT (121.360965275 25.04008085),"(281036.83386307984, 2770712.0812995993)",2875.276084,25.930513,0.952740,-0.303787,-2.449294e-16,1.000000
2,2905.662334,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,POINT (286539.710 2771997.554),2020-03-04,3.0,POINT (121.360965275 25.04008085),"(281036.83386307984, 2770712.0812995993)",2875.276084,25.930513,-0.356738,0.934204,-2.697968e-01,0.962917
3,300.946788,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,POINT (286792.895 2772160.239),2020-03-04,3.0,POINT (121.360965275 25.04008085),"(281036.83386307984, 2770712.0812995993)",2875.276084,25.930513,0.870530,-0.492115,-2.697968e-01,0.962917
4,460.535020,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,POINT (286762.953 2772619.799),2020-03-04,3.0,POINT (121.360965275 25.04008085),"(281036.83386307984, 2770712.0812995993)",2875.276084,25.930513,-0.136133,0.990691,-5.195840e-01,0.854419
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18162,0.846538,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,POINT (301328.622 2767590.969),2020-09-10,4.0,POINT (121.50853978 25.01551976),"(301329.39272735675, 2767593.8430754533)",1.168637,0.000004,0.262224,-0.965007,-3.984011e-01,-0.917211
18163,0.245558,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,POINT (301328.591 2767591.213),2020-09-10,4.0,POINT (121.50853978 25.01551976),"(301329.39272735675, 2767593.8430754533)",1.168637,0.000004,-0.265977,0.963979,-1.361666e-01,-0.990686
18164,1.246918,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,POINT (301327.777 2767590.268),2020-09-10,4.0,POINT (121.50853978 25.01551976),"(301329.39272735675, 2767593.8430754533)",1.168637,0.000004,0.998484,0.055036,-1.361666e-01,-0.990686
18165,0.000000,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,POINT (301327.777 2767590.268),2020-09-10,4.0,POINT (121.50853978 25.01551976),"(301329.39272735675, 2767593.8430754533)",1.168637,0.000004,0.000000,1.000000,-1.361666e-01,-0.990686


In [631]:
columns = ['date_part'] # 要操作的两列循环特征
for col in columns:
# 对每个循环特征派生两个特征列，如对于4月，是2pi*4/12，取sin和cos即可
    lstm_home_cyclic[col+'_sin'] = np.sin((2*np.pi*lstm_home_cyclic[col])/7.0)
    lstm_home_cyclic[col+'_cos'] = np.cos((2*np.pi*lstm_home_cyclic[col])/7.0)
lstm_home_cyclic=lstm_home_cyclic.drop(columns,axis=1) # 删除原来的循环特征列
lstm_home_cyclic

,distance,device_id_x,PK,geom,to_date_x,home,center,radius,km2,angle_sin,angle_cos,hr_sin,hr_cos,date_part_sin,date_part_cos
0,0.000000,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,POINT (285333.598 2767770.953),2020-03-04,POINT (121.360965275 25.04008085),"(281036.83386307984, 2770712.0812995993)",2875.276084,25.930513,NaN,NaN,-2.449294e-16,1.000000,0.433884,-0.900969
1,2176.072253,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,POINT (287030.214 2769133.592),2020-03-04,POINT (121.360965275 25.04008085),"(281036.83386307984, 2770712.0812995993)",2875.276084,25.930513,0.952740,-0.303787,-2.449294e-16,1.000000,0.433884,-0.900969
2,2905.662334,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,POINT (286539.710 2771997.554),2020-03-04,POINT (121.360965275 25.04008085),"(281036.83386307984, 2770712.0812995993)",2875.276084,25.930513,-0.356738,0.934204,-2.697968e-01,0.962917,0.433884,-0.900969
3,300.946788,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,POINT (286792.895 2772160.239),2020-03-04,POINT (121.360965275 25.04008085),"(281036.83386307984, 2770712.0812995993)",2875.276084,25.930513,0.870530,-0.492115,-2.697968e-01,0.962917,0.433884,-0.900969
4,460.535020,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,POINT (286762.953 2772619.799),2020-03-04,POINT (121.360965275 25.04008085),"(281036.83386307984, 2770712.0812995993)",2875.276084,25.930513,-0.136133,0.990691,-5.195840e-01,0.854419,0.433884,-0.900969
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18162,0.846538,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,POINT (301328.622 2767590.969),2020-09-10,POINT (121.50853978 25.01551976),"(301329.39272735675, 2767593.8430754533)",1.168637,0.000004,0.262224,-0.965007,-3.984011e-01,-0.917211,-0.433884,-0.900969
18163,0.245558,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,POINT (301328.591 2767591.213),2020-09-10,POINT (121.50853978 25.01551976),"(301329.39272735675, 2767593.8430754533)",1.168637,0.000004,-0.265977,0.963979,-1.361666e-01,-0.990686,-0.433884,-0.900969
18164,1.246918,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,POINT (301327.777 2767590.268),2020-09-10,POINT (121.50853978 25.01551976),"(301329.39272735675, 2767593.8430754533)",1.168637,0.000004,0.998484,0.055036,-1.361666e-01,-0.990686,-0.433884,-0.900969
18165,0.000000,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,POINT (301327.777 2767590.268),2020-09-10,POINT (121.50853978 25.01551976),"(301329.39272735675, 2767593.8430754533)",1.168637,0.000004,0.000000,1.000000,-1.361666e-01,-0.990686,-0.433884,-0.900969


In [632]:
lstm_home_cyclic=gpd.GeoDataFrame(lstm_home_cyclic, crs="EPSG:3826", geometry='geom')
lstm_home_cyclic.crs

<Projected CRS: EPSG:3826>
Name: TWD97 / TM2 zone 121
Axis Info [cartesian]:
- X[east]: Easting (metre)
- Y[north]: Northing (metre)
Area of Use:
- name: Taiwan - 120°E to 122°E
- bounds: (119.99, 20.41, 122.06, 26.72)
Coordinate Operation:
- name: Taiwan 2-degree TM zone 121
- method: Transverse Mercator
Datum: Taiwan Datum 1997
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

In [633]:
lstm_home_cyclic=lstm_home_cyclic.to_crs(crs="EPSG:4326")
lstm_home_cyclic

,distance,device_id_x,PK,geom,to_date_x,home,center,radius,km2,angle_sin,angle_cos,hr_sin,hr_cos,date_part_sin,date_part_cos
0,0.000000,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,POINT (121.35009 25.01758),2020-03-04,POINT (121.360965275 25.04008085),"(281036.83386307984, 2770712.0812995993)",2875.276084,25.930513,NaN,NaN,-2.449294e-16,1.000000,0.433884,-0.900969
1,2176.072253,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,POINT (121.36694 25.02985),2020-03-04,POINT (121.360965275 25.04008085),"(281036.83386307984, 2770712.0812995993)",2875.276084,25.930513,0.952740,-0.303787,-2.449294e-16,1.000000,0.433884,-0.900969
2,2905.662334,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,POINT (121.36216 25.05571),2020-03-04,POINT (121.360965275 25.04008085),"(281036.83386307984, 2770712.0812995993)",2875.276084,25.930513,-0.356738,0.934204,-2.697968e-01,0.962917,0.433884,-0.900969
3,300.946788,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,POINT (121.36467 25.05718),2020-03-04,POINT (121.360965275 25.04008085),"(281036.83386307984, 2770712.0812995993)",2875.276084,25.930513,0.870530,-0.492115,-2.697968e-01,0.962917,0.433884,-0.900969
4,460.535020,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,POINT (121.36439 25.06133),2020-03-04,POINT (121.360965275 25.04008085),"(281036.83386307984, 2770712.0812995993)",2875.276084,25.930513,-0.136133,0.990691,-5.195840e-01,0.854419,0.433884,-0.900969
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18162,0.846538,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,POINT (121.50857 25.01550),2020-09-10,POINT (121.50853978 25.01551976),"(301329.39272735675, 2767593.8430754533)",1.168637,0.000004,0.262224,-0.965007,-3.984011e-01,-0.917211,-0.433884,-0.900969
18163,0.245558,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,POINT (121.50857 25.01550),2020-09-10,POINT (121.50853978 25.01551976),"(301329.39272735675, 2767593.8430754533)",1.168637,0.000004,-0.265977,0.963979,-1.361666e-01,-0.990686,-0.433884,-0.900969
18164,1.246918,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,POINT (121.50856 25.01550),2020-09-10,POINT (121.50853978 25.01551976),"(301329.39272735675, 2767593.8430754533)",1.168637,0.000004,0.998484,0.055036,-1.361666e-01,-0.990686,-0.433884,-0.900969
18165,0.000000,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,POINT (121.50856 25.01550),2020-09-10,POINT (121.50853978 25.01551976),"(301329.39272735675, 2767593.8430754533)",1.168637,0.000004,0.000000,1.000000,-1.361666e-01,-0.990686,-0.433884,-0.900969


In [634]:
lstm_home_cyclic=lstm_home_cyclic.rename(columns={'geom':'geometry'})
lstm_home_cyclic=lstm_home_cyclic.rename(columns={'home':'geometry_2'})
lstm_home_cyclic

,distance,device_id_x,PK,geometry,to_date_x,geometry_2,center,radius,km2,angle_sin,angle_cos,hr_sin,hr_cos,date_part_sin,date_part_cos
0,0.000000,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,POINT (121.35009 25.01758),2020-03-04,POINT (121.360965275 25.04008085),"(281036.83386307984, 2770712.0812995993)",2875.276084,25.930513,NaN,NaN,-2.449294e-16,1.000000,0.433884,-0.900969
1,2176.072253,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,POINT (121.36694 25.02985),2020-03-04,POINT (121.360965275 25.04008085),"(281036.83386307984, 2770712.0812995993)",2875.276084,25.930513,0.952740,-0.303787,-2.449294e-16,1.000000,0.433884,-0.900969
2,2905.662334,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,POINT (121.36216 25.05571),2020-03-04,POINT (121.360965275 25.04008085),"(281036.83386307984, 2770712.0812995993)",2875.276084,25.930513,-0.356738,0.934204,-2.697968e-01,0.962917,0.433884,-0.900969
3,300.946788,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,POINT (121.36467 25.05718),2020-03-04,POINT (121.360965275 25.04008085),"(281036.83386307984, 2770712.0812995993)",2875.276084,25.930513,0.870530,-0.492115,-2.697968e-01,0.962917,0.433884,-0.900969
4,460.535020,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,POINT (121.36439 25.06133),2020-03-04,POINT (121.360965275 25.04008085),"(281036.83386307984, 2770712.0812995993)",2875.276084,25.930513,-0.136133,0.990691,-5.195840e-01,0.854419,0.433884,-0.900969
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18162,0.846538,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,POINT (121.50857 25.01550),2020-09-10,POINT (121.50853978 25.01551976),"(301329.39272735675, 2767593.8430754533)",1.168637,0.000004,0.262224,-0.965007,-3.984011e-01,-0.917211,-0.433884,-0.900969
18163,0.245558,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,POINT (121.50857 25.01550),2020-09-10,POINT (121.50853978 25.01551976),"(301329.39272735675, 2767593.8430754533)",1.168637,0.000004,-0.265977,0.963979,-1.361666e-01,-0.990686,-0.433884,-0.900969
18164,1.246918,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,POINT (121.50856 25.01550),2020-09-10,POINT (121.50853978 25.01551976),"(301329.39272735675, 2767593.8430754533)",1.168637,0.000004,0.998484,0.055036,-1.361666e-01,-0.990686,-0.433884,-0.900969
18165,0.000000,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,POINT (121.50856 25.01550),2020-09-10,POINT (121.50853978 25.01551976),"(301329.39272735675, 2767593.8430754533)",1.168637,0.000004,0.000000,1.000000,-1.361666e-01,-0.990686,-0.433884,-0.900969


In [636]:
lstm_home_cyclic=lstm_home_cyclic.set_geometry("geometry_2")
lstm_home_cyclic.info()
lstm_home_cyclic

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 18167 entries, 0 to 18166
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype   
---  ------         --------------  -----   
 0   distance       18167 non-null  float64 
 1   device_id_x    18167 non-null  object  
 2   PK             18167 non-null  object  
 3   geometry       18167 non-null  geometry
 4   to_date_x      18167 non-null  object  
 5   geometry_2     18167 non-null  geometry
 6   center         18167 non-null  object  
 7   radius         18167 non-null  float64 
 8   km2            18167 non-null  float64 
 9   angle_sin      17307 non-null  float64 
 10  angle_cos      17307 non-null  float64 
 11  hr_sin         18167 non-null  float64 
 12  hr_cos         18167 non-null  float64 
 13  date_part_sin  18167 non-null  float64 
 14  date_part_cos  18167 non-null  float64 
dtypes: float64(9), geometry(2), object(4)
memory usage: 2.8+ MB


,distance,device_id_x,PK,geometry,to_date_x,geometry_2,center,radius,km2,angle_sin,angle_cos,hr_sin,hr_cos,date_part_sin,date_part_cos
0,0.000000,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,POINT (121.35009 25.01758),2020-03-04,POINT (121.36097 25.04008),"(281036.83386307984, 2770712.0812995993)",2875.276084,25.930513,NaN,NaN,-2.449294e-16,1.000000,0.433884,-0.900969
1,2176.072253,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,POINT (121.36694 25.02985),2020-03-04,POINT (121.36097 25.04008),"(281036.83386307984, 2770712.0812995993)",2875.276084,25.930513,0.952740,-0.303787,-2.449294e-16,1.000000,0.433884,-0.900969
2,2905.662334,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,POINT (121.36216 25.05571),2020-03-04,POINT (121.36097 25.04008),"(281036.83386307984, 2770712.0812995993)",2875.276084,25.930513,-0.356738,0.934204,-2.697968e-01,0.962917,0.433884,-0.900969
3,300.946788,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,POINT (121.36467 25.05718),2020-03-04,POINT (121.36097 25.04008),"(281036.83386307984, 2770712.0812995993)",2875.276084,25.930513,0.870530,-0.492115,-2.697968e-01,0.962917,0.433884,-0.900969
4,460.535020,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,POINT (121.36439 25.06133),2020-03-04,POINT (121.36097 25.04008),"(281036.83386307984, 2770712.0812995993)",2875.276084,25.930513,-0.136133,0.990691,-5.195840e-01,0.854419,0.433884,-0.900969
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18162,0.846538,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,POINT (121.50857 25.01550),2020-09-10,POINT (121.50854 25.01552),"(301329.39272735675, 2767593.8430754533)",1.168637,0.000004,0.262224,-0.965007,-3.984011e-01,-0.917211,-0.433884,-0.900969
18163,0.245558,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,POINT (121.50857 25.01550),2020-09-10,POINT (121.50854 25.01552),"(301329.39272735675, 2767593.8430754533)",1.168637,0.000004,-0.265977,0.963979,-1.361666e-01,-0.990686,-0.433884,-0.900969
18164,1.246918,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,POINT (121.50856 25.01550),2020-09-10,POINT (121.50854 25.01552),"(301329.39272735675, 2767593.8430754533)",1.168637,0.000004,0.998484,0.055036,-1.361666e-01,-0.990686,-0.433884,-0.900969
18165,0.000000,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,POINT (121.50856 25.01550),2020-09-10,POINT (121.50854 25.01552),"(301329.39272735675, 2767593.8430754533)",1.168637,0.000004,0.000000,1.000000,-1.361666e-01,-0.990686,-0.433884,-0.900969


In [637]:
from geoalchemy2 import Geometry,WKTElement
server_ip='140.137.37.194'
engine = create_engine('postgresql://postgres:Co43270131@%s:5432/hyxen' %(server_ip), use_batch_mode=True)
lstm_home_cyclic['geom'] = lstm_home_cyclic['geometry'].apply(lambda x: WKTElement(x.wkt, srid=4326))
lstm_home_cyclic['geom_2'] = lstm_home_cyclic['geometry_2'].apply(lambda x: WKTElement(x.wkt, srid=4326))
lstm_home_cyclic.drop('geometry',1,inplace=True)
lstm_home_cyclic.drop('geometry_2',1,inplace=True)

In [638]:
lstm_home_cyclic

,distance,device_id_x,PK,to_date_x,center,radius,km2,angle_sin,angle_cos,hr_sin,hr_cos,date_part_sin,date_part_cos,geom,geom_2
0,0.000000,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,2020-03-04,"(281036.83386307984, 2770712.0812995993)",2875.276084,25.930513,NaN,NaN,-2.449294e-16,1.000000,0.433884,-0.900969,POINT (121.350094 25.0175849),POINT (121.360965275 25.04008085)
1,2176.072253,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,2020-03-04,"(281036.83386307984, 2770712.0812995993)",2875.276084,25.930513,0.952740,-0.303787,-2.449294e-16,1.000000,0.433884,-0.900969,POINT (121.3669408 25.0298465),POINT (121.360965275 25.04008085)
2,2905.662334,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,2020-03-04,"(281036.83386307984, 2770712.0812995993)",2875.276084,25.930513,-0.356738,0.934204,-2.697968e-01,0.962917,0.433884,-0.900969,POINT (121.3621563 25.0557147),POINT (121.360965275 25.04008085)
3,300.946788,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,2020-03-04,"(281036.83386307984, 2770712.0812995993)",2875.276084,25.930513,0.870530,-0.492115,-2.697968e-01,0.962917,0.433884,-0.900969,POINT (121.36467 25.0571773),POINT (121.360965275 25.04008085)
4,460.535020,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,+FW2xwkAeqaXGUX/flka5wsTCRu5y7h/633MJfX1IbYNAC...,2020-03-04,"(281036.83386307984, 2770712.0812995993)",2875.276084,25.930513,-0.136133,0.990691,-5.195840e-01,0.854419,0.433884,-0.900969,POINT (121.3643855 25.06132699999999),POINT (121.360965275 25.04008085)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18162,0.846538,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,2020-09-10,"(301329.39272735675, 2767593.8430754533)",1.168637,0.000004,0.262224,-0.965007,-3.984011e-01,-0.917211,-0.433884,-0.900969,POINT (121.5085658 25.0155024),POINT (121.50853978 25.01551976)
18163,0.245558,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,2020-09-10,"(301329.39272735675, 2767593.8430754533)",1.168637,0.000004,-0.265977,0.963979,-1.361666e-01,-0.990686,-0.433884,-0.900969,POINT (121.5085655 25.0155046),POINT (121.50853978 25.01551976)
18164,1.246918,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,2020-09-10,"(301329.39272735675, 2767593.8430754533)",1.168637,0.000004,0.998484,0.055036,-1.361666e-01,-0.990686,-0.433884,-0.900969,POINT (121.5085574 25.0154961),POINT (121.50853978 25.01551976)
18165,0.000000,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,ze7d9kp6c6w5/6Ooy/n0b/axzJelxBDFczqWr9d4yfznE2...,2020-09-10,"(301329.39272735675, 2767593.8430754533)",1.168637,0.000004,0.000000,1.000000,-1.361666e-01,-0.990686,-0.433884,-0.900969,POINT (121.5085574 25.0154961),POINT (121.50853978 25.01551976)


In [639]:
#lstm_home_cyclic.to_sql('lstm_home_cyclic', schema='chi', index=False, if_exists='append',con=engine)
lstm_home_cyclic.to_sql('lstm_home_cyclic', engine, if_exists='replace',index=False,schema='chi', dtype={'geom': Geometry('POINT', srid = 4326),'geom_2': Geometry('POINT', srid = 4326)})

### 沒有家的循環編碼

In [649]:
lstm_NOT_home_cyclic = lstm_NOT_home.copy()
columns = ['angle'] # 要操作的两列循环特征
for col in columns:
# 对每个循环特征派生两个特征列，如对于4月，是2pi*4/12，取sin和cos即可
    lstm_NOT_home_cyclic[col+'_sin'] = np.sin((2*np.pi*lstm_NOT_home_cyclic[col])/180)
    lstm_NOT_home_cyclic[col+'_cos'] = np.cos((2*np.pi*lstm_NOT_home_cyclic[col])/180)
lstm_NOT_home_cyclic=lstm_NOT_home_cyclic.drop(columns,axis=1) # 删除原来的循环特征列

In [650]:
columns = ['hr'] # 要操作的两列循环特征
for col in columns:
# 对每个循环特征派生两个特征列，如对于4月，是2pi*4/12，取sin和cos即可
    lstm_NOT_home_cyclic[col+'_sin'] = np.sin((2*np.pi*lstm_NOT_home_cyclic[col])/23.0)
    lstm_NOT_home_cyclic[col+'_cos'] = np.cos((2*np.pi*lstm_NOT_home_cyclic[col])/23.0)
lstm_NOT_home_cyclic=lstm_NOT_home_cyclic.drop(columns,axis=1) # 删除原来的循环特征列

In [651]:
columns = ['date_part'] # 要操作的两列循环特征
for col in columns:
# 对每个循环特征派生两个特征列，如对于4月，是2pi*4/12，取sin和cos即可
    lstm_NOT_home_cyclic[col+'_sin'] = np.sin((2*np.pi*lstm_NOT_home_cyclic[col])/7.0)
    lstm_NOT_home_cyclic[col+'_cos'] = np.cos((2*np.pi*lstm_NOT_home_cyclic[col])/7.0)
lstm_NOT_home_cyclic=lstm_NOT_home_cyclic.drop(columns,axis=1) # 删除原来的循环特征列
#lstm_NOT_home_cyclic

In [652]:
lstm_NOT_home_cyclic=gpd.GeoDataFrame(lstm_NOT_home_cyclic, crs="EPSG:3826", geometry='geom')
lstm_NOT_home_cyclic.crs

<Projected CRS: EPSG:3826>
Name: TWD97 / TM2 zone 121
Axis Info [cartesian]:
- X[east]: Easting (metre)
- Y[north]: Northing (metre)
Area of Use:
- name: Taiwan - 120°E to 122°E
- bounds: (119.99, 20.41, 122.06, 26.72)
Coordinate Operation:
- name: Taiwan 2-degree TM zone 121
- method: Transverse Mercator
Datum: Taiwan Datum 1997
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

In [653]:
lstm_NOT_home_cyclic=lstm_NOT_home_cyclic.to_crs(crs="EPSG:4326")
#lstm_NOT_home_cyclic

In [654]:
lstm_NOT_home_cyclic=lstm_NOT_home_cyclic.rename(columns={'geom':'geometry'})
#lstm_NOT_home_cyclic=lstm_NOT_home_cyclic.rename(columns={'home':'geometry_2'})

In [656]:
lstm_NOT_home_cyclic['geom'] = lstm_NOT_home_cyclic['geometry'].apply(lambda x: WKTElement(x.wkt, srid=4326))
#lstm_NOT_home_cyclic['geom_2'] = lstm_NOT_home_cyclic['geometry_2'].apply(lambda x: WKTElement(x.wkt, srid=4326))
lstm_NOT_home_cyclic.drop('geometry',1,inplace=True)
#lstm_NOT_home_cyclic.drop('geometry_2',1,inplace=True)

C:\Users\GIS519\Anaconda3\envs\python38\lib\site-packages\geopandas\geodataframe.py:852: UserWarning: Geometry column does not contain geometry.
  warnings.warn("Geometry column does not contain geometry.")


In [657]:
lstm_NOT_home_cyclic.to_sql('lstm_NOT_home_cyclic', engine, if_exists='replace',index=False,schema='chi', dtype={'geom': Geometry('POINT', srid = 4326),'geom_2': Geometry('POINT', srid = 4326)})

----------------------------------------------------------------

In [423]:
# normalize the dataset
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction import DictVectorizer

pd.DataFrame(lstm_taipei_allurser_distance.toarray(), columns=vec.get_feature_names())

AttributeError: 'GeoDataFrame' object has no attribute 'toarray'

In [188]:
# Adding the first LSTM layer and some Dropout regularisation
regressor = Sequential()

regressor.add(LSTM(units = 50, return_sequences = True, input_shape = (X_train.shape[1], 1)))
regressor.add(Dropout(0.2))

# Adding a second LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))

# Adding a third LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))

# Adding a fourth LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 50))
regressor.add(Dropout(0.2))

NameError: name 'X_train' is not defined